In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import gensim

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
data_root = '/home/thenhz/workspace/hate-speech/data/'

GLOVE_MODEL_FILE = data_root + 'word2vec/glove_WIKI'


word2vec_model = gensim.models.Word2Vec.load(GLOVE_MODEL_FILE)
print("word2vec model loaded...")

word2vec model loaded...


In [9]:
from nltk.corpus import stopwords

nltk_stopwords = stopwords.words('italian')

data = pd.read_csv('data/hate-speech/haspeede_TW-train.tsv', sep='\t', names=['text','sentiment'])
print("data file loaded..")


FLAGS = re.MULTILINE | re.DOTALL

#preprocess

def preprocess_tweet(text, stopwords, FLAGS = FLAGS):
# Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"
    
    def hashtag(text):
        text = text.group()
        hashtag_body = text[1:]
        return hashtag_body
    
    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "")
    text = re_sub(r"/","")
    text = re_sub(r"@\w+", "")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<NHZSMILE>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<NHZLOLFACE>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<NHZSADFACE>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<NHZNEUTRALFACE>")
    text = re_sub(r"<3","<NHZ_HEART>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<NHZNUMBERS>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", "")
    #text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <NHZ_ELONG>")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    #text = re_sub(r"([A-Z]){2,}", allcaps)
    
    #text = ''.join([c for c in text if c not in punctuation])
    
    #remove punctuation
    #text_no_punct = ''
    #for c in text:
    #    if c not in punctuation:
    #        text_no_punct += c
    #    else:
    #        text_no_punct += ' '
    #text = text_no_punct
    
    words = re.findall(r'[a-z]+', text.lower())
    text = ' '.join(words)
    
    
    #remove stopwords
    words = text.split()
    words = [word for word in words if word not in stopwords]
    text = ' '.join(words)
    
    return text


def filter_stop_words(train_sentences, stopwords):
    count = 0
    for i, sentence in train_sentences.iterrows():
        #print(sentence['text'])
        #if(count>10):
        #    break
        #count+=1
        new_sent = preprocess_tweet(sentence['text'],stopwords)
        #print(new_sent)
        #new_sent = ' '.join([word for word in new_sent.split() if word not in stop_words])
        #print(new_sent)
        train_sentences.set_value(i, 'text', new_sent, takeable=False)
    return train_sentences

train_sentences = filter_stop_words(data, nltk_stopwords)

data file loaded..


/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:78: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [10]:
train_sentences

,text,sentiment
576268275560116224,invasione animali forse offesa animali stranie...,1
815833978691260416,terrorismo mettere stato soggezione persone re...,0
815112299044372480,infatti finch guadagnato campi rom ok alemanno...,0
577451440047853568,piovegovernolad italia stranieri culo va bene ...,1
844921803575382016,londonattack chiedete buonisti cavolo cosa vog...,1
811099342593462272,corriere tangenti mafia capitale dimenticatama...,0
833323806873833473,perch quando migranti israeliti arrivarono ter...,0
835493194796314624,divertimento giorno trovare patrioti italiani ...,0
839605324684361730,modena comune paga benzina nomadi portano figl...,0
851906322283016192,altro islam cristianesimo dobbiamo sorbire ign...,0


In [11]:
#data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 0].size) #positive
print(data[ data['sentiment'] == 1].size) #negative



4052
1944


In [12]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/keras_preprocessing/text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [13]:
len(tokenizer.word_index)

8353

In [54]:
word_vectors = word2vec_model.wv
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 100


In [55]:
len(word_vectors.vocab)

730613

In [56]:
WV_DIM = 100

# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(max_fatures, WV_DIM) - 0.5) / 5.0
for word, i in tokenizer.word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass 

In [57]:
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, WV_DIM,weights=[wv_matrix], input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out,return_sequences=True, dropout_U=0.2, dropout_W=0.2))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(8))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])
print(model.summary())

#model = Sequential()
#model.add(LSTM(32, return_sequences=True,
#               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32))  # return a single vector of dimension 32
#model.add(Dense(10, activation='softmax'))

#model.compile(loss='categorical_crossentropy',
#              optimizer='rmsprop',
#              metrics=['accuracy'])

/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
/home/thenhz/anaconda3/envs/nlp36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 21, 100)           200000    
_________________________________________________________________
lstm_12 (LSTM)               (None, 21, 196)           232848    
_________________________________________________________________
lstm_13 (LSTM)               (None, 21, 32)            29312     
_________________________________________________________________
lstm_14 (LSTM)               (None, 8)                 1312      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 18        
Total params: 463,490
Trainable params: 463,490
Non-trainable params: 0
_________________________________________________________________
None


In [58]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2008, 21) (2008, 2)
(990, 21) (990, 2)


In [60]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, validation_split = 0.1, batch_size=batch_size, verbose = 2)

Train on 1807 samples, validate on 201 samples
Epoch 1/5
 - 3s - loss: 0.1358 - acc: 0.9552 - val_loss: 0.9848 - val_acc: 0.7015
Epoch 2/5
 - 3s - loss: 0.1162 - acc: 0.9629 - val_loss: 1.1432 - val_acc: 0.6965
Epoch 3/5
 - 3s - loss: 0.1079 - acc: 0.9646 - val_loss: 1.1318 - val_acc: 0.6965
Epoch 4/5
 - 3s - loss: 0.0930 - acc: 0.9707 - val_loss: 1.1718 - val_acc: 0.6915
Epoch 5/5
 - 4s - loss: 0.0881 - acc: 0.9745 - val_loss: 1.2393 - val_acc: 0.6567


In [61]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score = model.evaluate(X_test, Y_test, batch_size = batch_size)
#logger.info('mse=%f, mae=%f, mape=%f' % (scores[0],scores[1],scores[2]))
#print(score)
#print("score: %.2f " % (score))
#print("acc: %.2f" % (acc))
print(score)

[]


In [62]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 59.86842105263158 %
neg_acc 78.134110787172 %


Parameter tuning